### DATASET CLEANING

In [51]:
## UNIFICATION OF ALL THE CSV FILES AND FULL DATASET LOAD

import pandas as pd
import csv
import glob

#Read all the files in the directory
file_paths = glob.glob("/Users/francescavasta/Desktop/ADSEM/II YEAR I SEM/UE 1 - Advanced programming/module 2 DATA VISUALIZATION/annonces csv/*.csv")

# Unify all the CSV in a unique DF
dfs_linkedin = [pd.read_csv(file) for file in file_paths]
merged_dfs_linkedin = pd.concat(dfs_linkedin, ignore_index = False) #only possible if all the csv have same number of columns, same name same order

#Save the result in a unique file CSV
merged_dfs_linkedin.to_csv("merged_annonces.csv",index= False)
print("created merged CSV")
print(merged_dfs_linkedin.size)

created merged CSV
5952


In [52]:
print(len(merged_dfs_linkedin))

496


In [53]:
## DATA MANIPULATION AND CLEANING 

merged_dfs_linkedin[["Type of Accomodation", "Address"]] = merged_dfs_linkedin["Type of accommodation and address"].str.split('-',1, expand = True)
merged_dfs_linkedin = merged_dfs_linkedin.drop(columns =["Type of accommodation and address"])

col_to_move = merged_dfs_linkedin.pop("Type of Accomodation")
merged_dfs_linkedin.insert(1,"Type of Accomodation",col_to_move)

col_to_move = merged_dfs_linkedin.pop("Address")
merged_dfs_linkedin.insert(2,"Address", col_to_move)
#check the result
print(merged_dfs_linkedin.head())

#save the result in a new csv file
merged_dfs_linkedin.to_csv("cleaned_annonces.csv", index = False)

                                                 URL  \
0  https://www.airbnb.fr/rooms/120514970925836693...   
1  https://www.airbnb.fr/rooms/35018780?adults=1&...   
2  https://www.airbnb.fr/rooms/104841383201487965...   
3  https://www.airbnb.fr/rooms/43532792?adults=1&...   
4  https://www.airbnb.fr/rooms/118375854256253325...   

                Type of Accomodation                   Address    Travelers  \
0          Logement entier : cabane            Traînel, France  2 voyageurs   
1                            Grotte    Noyers-sur-Cher, France  4 voyageurs   
2     Logement entier : appartement              Rouen, France  2 voyageurs   
3  Logement entier : maison d'hôtes              Ussel, France  2 voyageurs   
4     Logement entier : hébergement       Valras-Plage, France  2 voyageurs   

         Rooms                        Bathrooms                 Rating  \
0  · 1 chambre                · 1 salle de bain  4,97 étoile(s) sur 5.   
1  · 1 chambre                · 1 salle 

/var/folders/ls/mj9skg6x7j38ls1ncwhc9s400000gn/T/ipykernel_53864/3476901757.py:3: FutureWarning: In a future version of pandas all arguments of StringMethods.split except for the argument 'pat' will be keyword-only.
  merged_dfs_linkedin[["Type of Accomodation", "Address"]] = merged_dfs_linkedin["Type of accommodation and address"].str.split('-',1, expand = True)


In [26]:
#IDENTIFICATION OF THE REGIONS

!pip install geopy


In [54]:
from geopy.geocoders import Nominatim
import time 

#geocoder confguration 
geolocator = Nominatim(user_agent="city_analysis")

def get_region(city):
    try:
        time.sleep(1)
        location = geolocator.geocode(city, exactly_one=True)
        if location:
            # Usa display_name per ottenere dettagli sulla regione
            display_name = location.raw.get('display_name', '')
            # Suddividi i dati e cerca la regione
            parts = display_name.split(', ')
            if len(parts) > 2:
                return parts[2:4]  # Ad esempio, 'Grand Est'
            else:
                return 'Unknown'
        else:
            print(f"Impossibile trovare la città: {city}")
            return 'Unknown'
    except Exception as e:
        print(f"Errore con la città {city}: {e}")
        return 'Unknown'

In [72]:
# Applica la funzione alla colonna Address
merged_dfs_linkedin['Region'] = merged_dfs_linkedin['Address'].apply(lambda x: get_region(x) if pd.notnull(x) else 'Unknown')

# Salva il file aggiornato
merged_dfs_linkedin.to_csv("with_regions.csv", index=False)


KeyboardInterrupt: 

In [48]:
city = "Rouen, France"
location = geolocator.geocode(city, exactly_one=True)
if location:
    print(f"Risultato: {location.raw}")
else:
    print("Città non trovata.")


Risultato: {'place_id': 90829492, 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. http://osm.org/copyright', 'osm_type': 'relation', 'osm_id': 75628, 'lat': '49.4404591', 'lon': '1.0939658', 'class': 'boundary', 'type': 'administrative', 'place_rank': 16, 'importance': 0.6847400787060492, 'addresstype': 'city', 'name': 'Rouen', 'display_name': 'Rouen, Seine-Maritime, Normandie, France métropolitaine, France', 'boundingbox': ['49.4172001', '49.4652601', '1.0300648', '1.1521157']}


In [80]:
# EXTRACT THE REGION

merged_dfs_linkedin = pd.read_csv("with_regions.csv")


#  Lista delle regioni francesi
french_regions = [
    'Auvergne-Rhône-Alpes', 'Bretagne', 'Centre-Val de Loire', 'Corse', 'Grand Est', 
    'Hauts-de-France', 'Île-de-France', 'Normandie', 'Nouvelle-Aquitaine', 
    'Occitanie', 'Pays de la Loire', 'Provence-Alpes-Côte d\'Azur', 'Bourgogne-Franche-Comté'
]

merged_dfs_linkedin['Region'].astype(str)
merged_dfs_linkedin['Region'].dtype

import numpy as np 

def extract_region(text):
    for region in french_regions:
        if region in text:
            return region  
    return np.nan

merged_dfs_linkedin['Extracted Region'] = merged_dfs_linkedin['Region'].apply(extract_region)


26

In [ ]:
merged_dfs_linkedin['Extracted Region'].head(20)

merged_dfs_linkedin['Extracted Region'].isna().sum()

In [82]:
#show the rows with na
rows_with_na = merged_dfs_linkedin[merged_dfs_linkedin['Extracted Region'].isna()][['Region', 'Extracted Region']]

# Visualizza il risultato
print(rows_with_na)

                                           Region Extracted Region
13                    ['Rouen', 'Seine-Maritime']              NaN
59                                        Unknown              NaN
83                                        Unknown              NaN
108                       ['Cherbourg', 'Manche']              NaN
144    ['6354', 'Schweiz/Suisse/Svizzera/Svizra']              NaN
145                ['Métropole de Lyon', 'Rhône']              NaN
147    ['8260', 'Schweiz/Suisse/Svizzera/Svizra']              NaN
148                                       Unknown              NaN
149    ['6078', 'Schweiz/Suisse/Svizzera/Svizra']              NaN
152    ['6078', 'Schweiz/Suisse/Svizzera/Svizra']              NaN
153                          ['Liguria', '16031']              NaN
158                        ['Lombardia', '22020']              NaN
162                       ['Lleida', 'Catalunya']              NaN
164    ['6354', 'Schweiz/Suisse/Svizzera/Svizra']             

In [84]:
####### CLEANING THE TEXT OF REVIEWS

import emoji 

def remove_emoji (text):
    return emoji.replace_emoji(text, replace = '')

merged_dfs_linkedin['Comments cleaned'] = merged_dfs_linkedin['Comments'].apply(remove_emoji)

# Salva il risultato
merged_dfs_linkedin.to_csv(" 3 without_emojis.csv", index=False)



In [87]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import spacy

# Configura NLTK e spaCy
nltk.download('stopwords')
nltk.download('punkt')
stop_words = set(stopwords.words('french'))

#!python -m spacy download fr_core_news_sm
nlp = spacy.load('fr_core_news_sm')


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/francescavasta/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/francescavasta/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
def process_comments(comment):
    # Tokenizzazione
    words = word_tokenize(comment.lower())
    # Rimuovi stop words
    filtered_words = [word for word in words if word not in stop_words and word.isalpha()]
    # Lemmatizzazione
    lemmatized_words = [nlp(word)[0].lemma_ for word in filtered_words]
    return lemmatized_words

# Applica la funzione ai commenti
merged_dfs_linkedin['Processed Comments'] = merged_dfs_linkedin['Comments'].apply(lambda x: process_comments(x) if pd.notnull(x) else [])

# Salva il risultato
merged_dfs_linkedin.to_csv("5 processed_comments.csv", index=False)

In [156]:
import pandas as pd

path = '/Users/francescavasta/Desktop/ADSEM/II YEAR I SEM/UE 1 - Advanced programming/module 2 DATA VISUALIZATION/annonces csv/datasets/5 processed_comments.csv'
data = pd.read_csv(path)

In [157]:
# Define the key columns to check
key_columns = ['URL', 'Address', 'Type of Accomodation', 'Host']

# Find duplicates based on the key columns
duplicates_key = data[data.duplicated(subset=key_columns)]

# Display the total number of duplicates for these key columns
print("Total duplicates (key columns):", len(duplicates_key))

# Display the duplicates for these key columns
print(duplicates_key)


Total duplicates (key columns): 0
Empty DataFrame
Columns: [URL, Type of Accomodation, Address, Travelers, Rooms, Bathrooms, Rating, Number of comments, Price per night, Free cancellation, Host, Host experience, Comments, Region, Extracted Region, Comments cleaned, Processed Comments]
Index: []


In [158]:
# Check for missing values in each column
missing_data = data.isnull().sum()

# Calculate the percentage of missing values
missing_percentage = (data.isnull().sum() / len(data)) * 100

# Display a summary
missing_summary = pd.DataFrame({
    "Missing Values": missing_data,
    "Percentage (%)": missing_percentage
}).sort_values(by="Missing Values", ascending=False)

print(missing_summary)

                      Missing Values  Percentage (%)
Extracted Region                  26        5.241935
Number of comments                 9        1.814516
Address                            2        0.403226
URL                                0        0.000000
Host                               0        0.000000
Comments cleaned                   0        0.000000
Region                             0        0.000000
Comments                           0        0.000000
Host experience                    0        0.000000
Price per night                    0        0.000000
Free cancellation                  0        0.000000
Type of Accomodation               0        0.000000
Rating                             0        0.000000
Bathrooms                          0        0.000000
Rooms                              0        0.000000
Travelers                          0        0.000000
Processed Comments                 0        0.000000


In [159]:
data = data.dropna()

print(data.shape)

(462, 17)


In [160]:
# Display unique values in Rooms
print(data['Rooms'].unique())


['· 1 chambre' '· Studio' '· 2 chambres' '· 3 chambres' '· 3 lits'
 '· 4 chambres' '· 5 chambres' '· 8 chambres' '· 1 lit']


In [161]:
# Display unique values in Rooms
print(data['Rooms'].value_counts())

· 1 chambre     169
· 2 chambres    159
· 3 chambres     99
· Studio         23
· 4 chambres      4
· 1 lit           3
· 3 lits          2
· 5 chambres      2
· 8 chambres      1
Name: Rooms, dtype: int64


## ROOMS

In [162]:
# Function to determine the type of accommodation
def extract_room_type(room_desc):
    if pd.isna(room_desc) or 'not specified' in room_desc.lower():
        return 'Unknown'
    elif 'studio' in room_desc.lower():
        return 'Studio'
    elif 'chambre' in room_desc.lower():
        return 'Room'
    else:
        return 'Unknown'

# Function to extract the number of rooms
def extract_number_of_rooms(room_desc):
    if pd.isna(room_desc) or 'not specified' in room_desc.lower():
        return None
    elif 'studio' in room_desc.lower():
        return 0
    elif 'chambre' in room_desc.lower():
        # Extract the first digit corresponding to the number of rooms
        match = pd.Series(room_desc).str.extract(r'(\d+)')
        return int(match[0].iloc[0]) if not match.empty and pd.notna(match[0].iloc[0]) else None
    else:
        return None

# Apply the functions
data['Room Type'] = data['Rooms'].apply(extract_room_type)
data['Number of Rooms'] = data['Rooms'].apply(extract_number_of_rooms)

# Check the results
print(data[['Rooms', 'Room Type', 'Number of Rooms']].head())


         Rooms Room Type  Number of Rooms
0  · 1 chambre      Room              1.0
1  · 1 chambre      Room              1.0
2  · 1 chambre      Room              1.0
3  · 1 chambre      Room              1.0
4  · 1 chambre      Room              1.0


In [163]:
print(data['Room Type'].unique())

['Room' 'Studio' 'Unknown']


In [164]:
print(data['Room Type'].value_counts())

Room       434
Studio      23
Unknown      5
Name: Room Type, dtype: int64


In [165]:
print(data['Number of Rooms'].describe())

count    457.000000
mean       1.789934
std        0.948029
min        0.000000
25%        1.000000
50%        2.000000
75%        2.000000
max        8.000000
Name: Number of Rooms, dtype: float64


In [166]:
data['Number of Rooms'] = data['Number of Rooms'].fillna(0).astype(int)

In [167]:
print(data['Host experience'].value_counts())

Superhôte · Hôte depuis 2 ans                       63
Superhôte · Hôte depuis 3 ans                       53
Superhôte · Hôte depuis 4 ans                       37
Superhôte · Hôte depuis 6 ans                       30
Superhôte · Hôte depuis 5 ans                       28
Superhôte · Hôte depuis 1 an                        25
Superhôte · Hôte depuis 7 ans                       19
Superhôte · Hôte depuis 8 ans                       19
Superhôte · 6 mois d'expérience en tant qu'hôte     16
Hôte depuis 7 ans                                   14
Superhôte · 5 mois d'expérience en tant qu'hôte     12
Superhôte · Hôte depuis 9 ans                       11
Hôte depuis 5 ans                                   11
Superhôte · 10 mois d'expérience en tant qu'hôte    10
Superhôte · Hôte depuis 10 ans                      10
1 mois d'expérience en tant qu'hôte                  9
Hôte depuis 2 ans                                    9
Superhôte · 7 mois d'expérience en tant qu'hôte      7
Hôte depui

In [168]:
# Function to clean the price per night
def clean_price(price_desc):
    if pd.isna(price_desc) or price_desc.lower() in ['not specified']:
        return None
    try:
        return float(price_desc.replace('€', '').strip())
    except ValueError:
        return None

# Function to clean the number of comments
def clean_comments(comment_desc):
    if pd.isna(comment_desc):
        return None
    try:
        return int(pd.Series(comment_desc).str.extract(r'(\d+)')[0].iloc[0])
    except (ValueError, IndexError):
        return None

# Function to clean the number of bathrooms
def clean_bathrooms(bathroom_desc):
    if pd.isna(bathroom_desc):
        return None
    try:
        match = pd.Series(bathroom_desc).str.extract(r'(\d+)')
        return int(match[0].iloc[0]) if not match.empty and pd.notna(match[0].iloc[0]) else None
    except (ValueError, IndexError):
        return None

# Apply the cleaning functions
data['Price per night'] = data['Price per night'].apply(clean_price)
data['Number of comments'] = data['Number of comments'].apply(clean_comments)
data['Bathrooms'] = data['Bathrooms'].apply(clean_bathrooms)

# Check the cleaned columns
print(data[['Price per night', 'Number of comments', 'Bathrooms']].head())


   Price per night  Number of comments  Bathrooms
0            115.0                61.0        1.0
1            152.0               417.0        1.0
2            120.0               165.0        1.0
3             32.0               457.0        1.0
4            207.0                84.0        2.0


In [169]:
# Function to clean the type of accomodation, applied only to the cells that contain 'Logement entier :'
def clean_accomodation_type(room_desc):
    if isinstance(room_desc, str) and 'Logement entier :' in room_desc:
        # Split the string after ': ' and take the last part
        return room_desc.split(': ')[-1]
    return room_desc  # Return the original value if 'Logement entier :' is not present

# Function to clean the type of accomodation, applied only to the cells that contain 'Logement entier :'
def extract_host_name(text):
    if isinstance(text, str) and 'Hôte :' in text:
        # Split the string after ': ' and take the last part
        return text.split(': ')[-1]
    return text  # Return the original value if 'Logement entier :' is not present

# Function to clean the host experience including "depuis" if present
def extract_host_experience(text):
    if isinstance(text, str):
        # Regex to extract "depuis" and the duration
        match = pd.Series(text).str.extract(r'((?:depuis\s*)?\d+\s(?:mois|ans))')
        
        # Extract the full match (e.g., "depuis 10 ans" or "5 mois")
        return match[0].iloc[0] if not match.empty and pd.notna(match[0].iloc[0]) else None
    return None

# Function to clean the host experience including "depuis" if present
def annulation_gratuite(text):
    if pd.notna(text) and 'Annulation gratuite' in text:
        return 'yes'
    else: 
        return 'no'

# apply the function 
data['Type of Accomodation'] = data['Type of Accomodation'].apply(clean_accomodation_type)
data['Host'] = data['Host'].apply(extract_host_name)
data['Host experience'] = data['Host experience'].apply(extract_host_experience)
data['Free cancellation cleaned'] = data['Free cancellation'].apply(annulation_gratuite)


In [170]:
# Check the cleaned columns
print(data[['Type of Accomodation', 'Host', 'Host experience', 'Free cancellation', 'Free cancellation cleaned']].head())

  Type of Accomodation                Host Host experience  \
0              cabane               Julien          5 mois   
1              Grotte             Sandrine   depuis 10 ans   
2         appartement               Claire    depuis 7 ans   
3      maison d'hôtes           Christophe    depuis 5 ans   
4         hébergement   Amandine Et Maxime    depuis 3 ans   

                       Free cancellation Free cancellation cleaned  
0   Annulation gratuite avant le 21 janv                       yes  
1                          Not specified                        no  
2                   Claire est Superhôte                        no  
3   Annulation gratuite avant le 11 janv                       yes  
4  Annulation gratuite pendant 48 heures                       yes  


In [171]:
# Function to clean the number of travelers
def clean_travelers(text):
    if pd.isna(text):
        return None
    try:
        match = pd.Series(text).str.extract(r'(\d+)')
        return int(match[0].iloc[0]) if not match.empty and pd.notna(match[0].iloc[0]) else None
    except (ValueError, IndexError):
        return None
    
# Function to clean the rating and extract the decimal part
def clean_rating(text):
    if pd.isna(text):  # Check if the value is NaN
        return None
    try:
        # Extract the decimal number with a comma
        match = pd.Series(text).str.extract(r'(\d+,\d+ |\d+)')  # Regex to capture the decimal number
        # Convert to float by replacing the comma with a dot
        return float(match[0].iloc[0].replace(',', '.')) if not match.empty and pd.notna(match[0].iloc[0]) else None
    except (ValueError, IndexError):
        return None

#apply the function 
data['Travelers'] = data['Travelers'].apply(clean_travelers)
data['Rating'] = data['Rating'].apply(clean_rating)

In [172]:
data.columns

Index(['URL', 'Type of Accomodation', 'Address', 'Travelers', 'Rooms',
       'Bathrooms', 'Rating', 'Number of comments', 'Price per night',
       'Free cancellation', 'Host', 'Host experience', 'Comments', 'Region',
       'Extracted Region', 'Comments cleaned', 'Processed Comments',
       'Room Type', 'Number of Rooms', 'Free cancellation cleaned'],
      dtype='object')

In [173]:
columns_to_drop = ['URL', 'Rooms','Free cancellation','Region']
data = data.drop(columns=(columns_to_drop))

data.head()

,Type of Accomodation,Address,Travelers,Bathrooms,Rating,Number of comments,Price per night,Host,Host experience,Comments,Extracted Region,Comments cleaned,Processed Comments,Room Type,Number of Rooms,Free cancellation cleaned
0,cabane,"Traînel, France",2,1.0,4.97,61.0,115.0,Julien,5 mois,Ce logement fait partie des 5 % de logements p...,Grand Est,Ce logement fait partie des 5 % de logements p...,"['logement', 'faire', 'partie', 'logement', 'p...",Room,1,yes
1,Grotte,"Noyers-sur-Cher, France",4,1.0,4.82,417.0,152.0,Sandrine,depuis 10 ans,Ma copine et moi avons passe 3 jours dans cett...,Centre-Val de Loire,Ma copine et moi avons passe 3 jours dans cett...,"['copine', 'passer', 'jour', 'ce', 'caverne', ...",Room,1,no
2,appartement,"Rouen, France",2,1.0,4.93,165.0,120.0,Claire,depuis 7 ans,L'un des logements préférés des voyageurs sur ...,Normandie,L'un des logements préférés des voyageurs sur ...,"['logement', 'préférer', 'voyageur', 'airbnb',...",Room,1,no
3,maison d'hôtes,"Ussel, France",2,1.0,4.93,457.0,32.0,Christophe,depuis 5 ans,L'un des logements préférés des voyageurs sur ...,Nouvelle-Aquitaine,L'un des logements préférés des voyageurs sur ...,"['logement', 'préférer', 'voyageur', 'airbnb',...",Room,1,yes
4,hébergement,"Valras-Plage, France",2,2.0,4.93,84.0,207.0,Amandine Et Maxime,depuis 3 ans,L'un des logements préférés des voyageurs sur ...,Occitanie,L'un des logements préférés des voyageurs sur ...,"['logement', 'préférer', 'voyageur', 'airbnb',...",Room,1,yes


In [174]:
#reorder columns
cols = data.columns.tolist()

# extract the last tree columns that we want to move
last_three_cols = cols[-3:]
print(last_three_cols)

# new order of columns
new_order = cols[:3] + last_three_cols + cols[3:-3]

# Riordiniamo il DataFrame
data = data[new_order]

# Verifica il risultato
print(data.head())

['Room Type', 'Number of Rooms', 'Free cancellation cleaned']
  Type of Accomodation                   Address  Travelers Room Type  \
0              cabane            Traînel, France          2      Room   
1              Grotte    Noyers-sur-Cher, France          4      Room   
2         appartement              Rouen, France          2      Room   
3      maison d'hôtes              Ussel, France          2      Room   
4         hébergement       Valras-Plage, France          2      Room   

   Number of Rooms Free cancellation cleaned  Bathrooms  Rating  \
0                1                       yes        1.0    4.97   
1                1                        no        1.0    4.82   
2                1                        no        1.0    4.93   
3                1                       yes        1.0    4.93   
4                1                       yes        2.0    4.93   

   Number of comments  Price per night                Host Host experience  \
0                6

In [175]:
# Save the cleaned file
data.to_csv('cleaned_airbnb_data.csv', index=False, encoding='utf-8')
print("Cleaned data saved to 'cleaned_airbnb_data.csv'.")

Cleaned data saved to 'cleaned_airbnb_data.csv'.


In [176]:
data.isna().sum()

Type of Accomodation          0
Address                       0
Travelers                     0
Room Type                     0
Number of Rooms               0
Free cancellation cleaned     0
Bathrooms                     4
Rating                        9
Number of comments            1
Price per night               9
Host                          0
Host experience              34
Comments                      0
Extracted Region              0
Comments cleaned              0
Processed Comments            0
dtype: int64